<a href="https://colab.research.google.com/github/Jarmos-san/PumpItUp/blob/master/PumpItUp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [DrivenData Competition | Pump it Up: Data Mining the Water Table](https://www.drivendata.org/competitions/7/pump-it-up-data-mining-the-water-table/page/23/)

Goal - ***To predict the operating condition of a waterpoint for each record in the dataset.***

[Discussion Forums](https://community.drivendata.org/c/pump-it-up-data-mining-the-water-table)

---

Problem Description has 3 sections:

- Features
- Labels
- Submission Format

## Features:

- `TSHAmount` - **Total static head (amount water available to waterpoint)**
- `DateOfRecording` - **The date the row was entered**
- `Funder` - **Who funded the well**
- `GPSHeight` - **Altitude of the well**
- `Installer` - **Organization that installed the well**
- `Longitude` - **GPS coordinate**
- `Latitude` - **GPS coordinate**
- `WaterPointName` - **Name of the waterpoint if there is one**
- `PrivateNum` -
- `Basin` - **Geographic water basin**
- `SubVillage` - **Geographic location**
- `Region` - **Geographic location**
- `RegionCode` - **Geographic location (coded)**
- `DistrictCode` - **Geographic location (coded)**
- `LocalGovernmentArea` - **Geographic location**
- `Ward` - **Geographic location**
- `Population` - **Population around the well**
- `PublicMeeting` - **True/False**
- `RecordedBy` - **Group entering this row of data**
- `SchemeManager` - **Who operates the waterpoint**
- `SchemeName` - **Who operates the waterpoint**
- `Permit` - **If the waterpoint is permitted**
- `YearOfConstruction` - **Year the waterpoint was constructed**
- `ExtractionType` - **The kind of extraction the waterpoint uses**
- `ExtractionTypeGroup` - **The kind of extraction the waterpoint uses**
- `ExtractionTypeClass` - **The kind of extraction the waterpoint uses**
- `MangementType` - **How the waterpoint is managed**
- `ManagementGroup` - **How the waterpoint is managed**
- `OperationCosts` - **What the water costs**
- `PaymentType` - **What the water costs**
- `WaterQuality` - **The quality of the water**
- `Qualityroup` - **The quality of the water**
- `Quantity` - **The quantity of water**
- `QuantityGroup` - **The quantity of water**
- `Source` - **The source of the water**
- `SourceType` - **The source of the water**
- `SourceClass` - **The source of the water**
- `WaterPointType` - **The kind of waterpoint**
- `WaterPointTypeGroup` - **The kind of waterpoint**

## Labels:

- `functional` - **the waterpoint is operational and there are no repairs needed**
- `functional needs repair` - **the waterpoint is operational, but needs repairs**
- `non functional` - **the waterpoint is not operational**

## Submission Format:

id 	  | status_group |
------|--------------|
50785 |	functional   |
51630 |	functional   |
17168 |	functional   |
45559 |	functional   |
49871 |	functional   |

## Data Download:

File 	                                                                                                            | Description                                       |
------------------------------------------------------------------------------------------------------------------|---------------------------------------------------|
[Submission format](https://s3.amazonaws.com/drivendata/data/7/public/SubmissionFormat.csv)                       |	The format for submitting your predictions        |
[Test Set Values](https://s3.amazonaws.com/drivendata/data/7/public/702ddfc5-68cd-4d1d-a0de-f5f566f76d91.csv)     |	The independent variables that need predictions   |
[Training Set Labels](https://s3.amazonaws.com/drivendata/data/7/public/0bf8bc6e-30d0-4c50-956a-603fc693d966.csv) |	The dependent variable (status_group) for each of the rows in Training set values   |
[Training Set Values](https://s3.amazonaws.com/drivendata/data/7/public/4910797b-ee55-40a7-8668-10efd5c1b960.csv) | The independent variables for the training set    |

In [35]:
# Loading Pandas 0.25.3 into the system along with the required files
import pandas as pd
import numpy as np

# Checking the version for easier debugging at some point of time.
print(f'Pandas version: {pd.__version__}')
print(f'Numpy version: {np.__version__}')

Pandas version: 0.25.3
Numpy version: 1.17.4


In [0]:
# Load .csv file directly from the provided links
SubmissionFormat = pd.read_csv('https://s3.amazonaws.com/drivendata/data/7/public/SubmissionFormat.csv')
TestSet = pd.read_csv('https://s3.amazonaws.com/drivendata/data/7/public/702ddfc5-68cd-4d1d-a0de-f5f566f76d91.csv')
TrainLabels = pd.read_csv('https://s3.amazonaws.com/drivendata/data/7/public/0bf8bc6e-30d0-4c50-956a-603fc693d966.csv')
TrainSet = pd.read_csv('https://s3.amazonaws.com/drivendata/data/7/public/4910797b-ee55-40a7-8668-10efd5c1b960.csv')

In [0]:
# Checking for the number of variables in the Label and Training set
print(f'Size of Labels set: {TrainLabels.shape}')
print(f'Size of Training set: {TrainSet.shape}')

Size of Labels set: (59400, 2)
Size of Training set: (59400, 40)


In [0]:
# Checking the last 3 variables in the dataset for a rough visual idea of the features.
TrainLabels.tail(3)

,id,status_group
59397,37057,functional
59398,31282,functional
59399,26348,functional


In [0]:
# Checking the tail of the dataset for rough visual idea of the available features.
TrainSet.tail(3)

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
59397,37057,0.0,2011-04-11,NaN,0,NaN,34.017087,-8.750434,Mashine,0,Rufiji,Madungulu,Mbeya,12,7,Mbarali,Chimala,0,True,GeoData Consultants Ltd,VWC,NaN,False,0,swn 80,swn 80,handpump,vwc,user-group,pay monthly,monthly,fluoride,fluoride,enough,enough,machine dbh,borehole,groundwater,hand pump,hand pump
59398,31282,0.0,2011-03-08,Malec,0,Musa,35.861315,-6.378573,Mshoro,0,Rufiji,Mwinyi,Dodoma,1,4,Chamwino,Mvumi Makulu,0,True,GeoData Consultants Ltd,VWC,NaN,True,0,nira/tanira,nira/tanira,handpump,vwc,user-group,never pay,never pay,soft,good,insufficient,insufficient,shallow well,shallow well,groundwater,hand pump,hand pump
59399,26348,0.0,2011-03-23,World Bank,191,World,38.104048,-6.747464,Kwa Mzee Lugawa,0,Wami / Ruvu,Kikatanyemba,Morogoro,5,2,Morogoro Rural,Ngerengere,150,True,GeoData Consultants Ltd,VWC,NaN,True,2002,nira/tanira,nira/tanira,handpump,vwc,user-group,pay when scheme fails,on failure,salty,salty,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump


In [48]:
# Merging the Labels and Training set into a single Dataframe as well renaming the Column names properly.
data = TrainSet.merge(TrainLabels, on='id')
RenamedColumns = ['ID', 'TSHAmount', 'DateOfRecording', 'Funder', 'GPSHeight', 'Installer', 'Longitutde',
              'Latitude', 'WaterPointName', 'PrivateNum', 'Basin', 'SubVillage', 'Region', 'RegionCode',
              'DistrictCode', 'LocalGovernmentArea', 'Ward', 'Population', 'PublicMeeting', 'RecordedBy',
              'SchemeManagement', 'Schemename', 'Permit', 'ConstructionYear', 'ExtractionType', 
              'ExtractionTypeGroup', 'ExtractionTypeClass', 'Management', 'ManagementGroup', 'Payment', 
              'PaymentType', 'WaterQuality', 'QualityGroup', 'Quantity', 'QuantityGroup', 'Source', 
              'SourceType', 'SourceClass', 'WaterPointType', 'WaterPointTypeGroup', 'StatusGroup']
data.set_axis(RenamedColumns, axis='columns', inplace=True)
data.head()

,ID,TSHAmount,DateOfRecording,Funder,GPSHeight,Installer,Longitutde,Latitude,WaterPointName,PrivateNum,Basin,SubVillage,Region,RegionCode,DistrictCode,LocalGovernmentArea,Ward,Population,PublicMeeting,RecordedBy,SchemeManagement,Schemename,Permit,ConstructionYear,ExtractionType,ExtractionTypeGroup,ExtractionTypeClass,Management,ManagementGroup,Payment,PaymentType,WaterQuality,QualityGroup,Quantity,QuantityGroup,Source,SourceType,SourceClass,WaterPointType,WaterPointTypeGroup,StatusGroup
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,functional
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,NaN,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,non functional
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional


In [49]:
# Checking basic stats of the Dataframe
data.describe()

,ID,TSHAmount,GPSHeight,Longitutde,Latitude,PrivateNum,RegionCode,DistrictCode,Population,ConstructionYear
count,59400.000000,59400.000000,59400.000000,59400.000000,5.940000e+04,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000
mean,37115.131768,317.650385,668.297239,34.077427,-5.706033e+00,0.474141,15.297003,5.629747,179.909983,1300.652475
std,21453.128371,2997.574558,693.116350,6.567432,2.946019e+00,12.236230,17.587406,9.633649,471.482176,951.620547
min,0.000000,0.000000,-90.000000,0.000000,-1.164944e+01,0.000000,1.000000,0.000000,0.000000,0.000000
25%,18519.750000,0.000000,0.000000,33.090347,-8.540621e+00,0.000000,5.000000,2.000000,0.000000,0.000000
50%,37061.500000,0.000000,369.000000,34.908743,-5.021597e+00,0.000000,12.000000,3.000000,25.000000,1986.000000
75%,55656.500000,20.000000,1319.250000,37.178387,-3.326156e+00,0.000000,17.000000,5.000000,215.000000,2004.000000
max,74247.000000,350000.000000,2770.000000,40.345193,-2.000000e-08,1776.000000,99.000000,80.000000,30500.000000,2013.000000


In [50]:
# Replacing 0.0 values with Numpy NaN values. 
data['TSHAmount'].replace(0.0, np.nan, inplace=True)
data['ConstructionYear'].replace(0.0, np.nan, inplace=True)
data['Population'].replace(0.0, np.nan, inplace=True)
data.isnull().sum()

ID                         0
TSHAmount              41639
DateOfRecording            0
Funder                  3635
GPSHeight                  0
Installer               3655
Longitutde                 0
Latitude                   0
WaterPointName             0
PrivateNum                 0
Basin                      0
SubVillage               371
Region                     0
RegionCode                 0
DistrictCode               0
LocalGovernmentArea        0
Ward                       0
Population             21381
PublicMeeting           3334
RecordedBy                 0
SchemeManagement        3877
Schemename             28166
Permit                  3056
ConstructionYear       20709
ExtractionType             0
ExtractionTypeGroup        0
ExtractionTypeClass        0
Management                 0
ManagementGroup            0
Payment                    0
PaymentType                0
WaterQuality               0
QualityGroup               0
Quantity                   0
QuantityGroup 

In [40]:
# Check for column data types for easier grouping and data manipulation.
data.dtypes

id                         int64
amount_tsh               float64
date_recorded             object
funder                    object
gps_height                 int64
installer                 object
longitude                float64
latitude                 float64
wpt_name                  object
num_private                int64
basin                     object
subvillage                object
region                    object
region_code                int64
district_code              int64
lga                       object
ward                      object
population               float64
public_meeting            object
recorded_by               object
scheme_management         object
scheme_name               object
permit                    object
construction_year        float64
extraction_type           object
extraction_type_group     object
extraction_type_class     object
management                object
management_group          object
payment                   object
payment_ty

# Rough Overview

- The column names have been cleaned but there seems to be some duplicates which needs to be taken care of.
- `NaN` values in some of the features need to be taken care of too or dropped at a later stage after understanding the need to.
- Seems like there are a lot of string-type values in the dataset which could be transformed into possible categories.
- Understand & read up more on Feature Engineering.
- Do some feature engineering with the data, learn how to check for relevancy in the features and drop some columns accordingly.
- Create model and then **Profit**!!